In [7]:
seed  = 1
from transformers import set_seed, BertTokenizer, TFAutoModelForMaskedLM, AutoTokenizer, get_scheduler, BertForPreTraining, BertForMaskedLM, BertConfig
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from torch.optim import AdamW
import argparse, torch, datasets, ast, operator, gc, os
import torch.nn as nn
set_seed(seed)

In [8]:
# def score(sentence,model):
# #     if len(sentence)>128:
# #         sentence = sentence[:128]
#     with torch.no_grad():
#         model.eval()
#         tokenize_input = tokenizer.tokenize(sentence)
#         tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
#         sen_len = len(tokenize_input)
#         sentence_loss = 0.

#         for i, word in enumerate(tokenize_input):
#             # add mask to i-th character of the sentence
#             tokenize_input[i] = '[MASK]'
#             mask_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
            
#             mask_input=mask_input.to(device)

#             output = model(mask_input)

#             prediction_scores = output[0]
#             softmax = nn.Softmax(dim=0)
#             ps = softmax(prediction_scores[0, i]).log()
#             word_loss = ps[tensor_input[0, i]]
#             word_loss_new = word_loss.detach().cpu().numpy()
#             sentence_loss += word_loss.item()
#             tokenize_input[i] = word
#         ppl = np.exp(-sentence_loss/sen_len)
#         del output,mask_input,ps,word_loss, prediction_scores,tensor_input,tokenize_input
#         gc.collect()
#         torch.cuda.empty_cache()
#         return ppl
    
def compute_ppl(sentence,model):
#     if len(sentence)>128:
#         sentence = sentence[:128]
    tokenize_input = tokenizer.tokenize(sentence)
    candidates = []
    target_words = []
    for i in range(len(tokenize_input)):
        temp = tokenize_input.copy()
        # temp = tokenize_input.copy()[:i+1]
        target_words.append(temp[i])
        temp[i] = tokenizer.mask_token
        candidates.append(temp)

        
    # print(candidates)
    with torch.no_grad():
        model.eval()     
        sentence_loss = 0.

        for i, (candidate, target_word) in enumerate(zip(candidates,target_words)):
            
            target_word_id = tokenizer.convert_tokens_to_ids(target_word)
            tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(candidate)]).to(device)
            output = model(tensor_input)
            prediction_scores = output.logits
            softmax = nn.Softmax(dim=0)
            
            ps = softmax(prediction_scores[0, i]).log()
            word_loss = ps[target_word_id]
            word_loss_new = word_loss.detach().cpu().numpy()
            sentence_loss += word_loss_new.item()
        #     print(word_loss_new)
        

        # print(sentence_loss)
        # print(sentence_loss/len(candidates))
        ppl = np.exp(-sentence_loss/len(candidates))
        del output,ps,word_loss, prediction_scores,tensor_input,tokenize_input
        gc.collect()
        torch.cuda.empty_cache()
        return ppl

In [9]:
parser = argparse.ArgumentParser(description='search for best template according to dev set')
parser.add_argument('--max_len', default=512, type=int, help="max sequence length")
parser.add_argument('--batch_size', default=2, type=int, help="batch size")
parser.add_argument('--model', default='./models/my_roberta/', type=str, help="pretrained model")
parser.add_argument('--result_output_dir', default='./ppl_results/', type=str, help="output directory")
parser.add_argument('--tokenizer', default='./models/my_roberta/', type=str, help="tokenizer")
parser.add_argument('--task', default='douban', type=str, help="task name")
parser.add_argument('--datasets', default='./datasets/', type=str, help="dataset dir")
args = parser.parse_args(args=[])

print(args)

Namespace(batch_size=2, datasets='./datasets/', max_len=512, model='./models/my_roberta/', result_output_dir='./ppl_results/', task='douban', tokenizer='./models/my_roberta/')


In [10]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'loading tokenizer {args.tokenizer}')
tokenizer = BertTokenizer.from_pretrained(f'{args.tokenizer}')
pretrained_model = BertForMaskedLM.from_pretrained(args.model).to(device)

loading tokenizer ./models/my_roberta/


Some weights of the model checkpoint at ./models/my_roberta/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
pd_all = pd.read_csv(f'{args.datasets}{args.task}_output.csv',names=['movie_name','scores','text','labels'],header=0)
texts = pd_all.text.tolist()

In [12]:
ppl_scores = []
for text in tqdm(texts):
    score = compute_ppl(text,pretrained_model)
    ppl_scores.append(score)

pd_all['ppl'] = ppl_scores
pd_all.to_csv(f'{args.datasets}{args.task}_ppl.csv')

100%|██████████| 1094/1094 [04:54<00:00,  3.72it/s]
